In [1]:
import xml.etree.ElementTree as ET
import os

def convert_box(size, box):
    dw = 1.0 / size[0]  # Width normalization
    dh = 1.0 / size[1]  # Height normalization
    x = (box[0] + box[2]) / 2.0  # Center x
    y = (box[1] + box[3]) / 2.0  # Center y
    w = box[2] - box[0]  # Width
    h = box[3] - box[1]  # Height
    return (x * dw, y * dh, w * dw, h * dh)

# Parse XML
tree = ET.parse("D:/pedri-dataset/annotations/annotations.xml")
root = tree.getroot()

output_dir = "D:/pedri-dataset/labels"
os.makedirs(output_dir, exist_ok=True)

for image in root.findall("image"):
    img_id = image.get("id")
    img_name = image.get("name").split("/")[-1].replace(".jpg", "")
    width = float(image.get("width"))
    height = float(image.get("height"))

    with open(f"{output_dir}/{img_name}.txt", "w") as f:
        for box in image.findall("box"):
            label = 0 if box.get("label") == "Pedri" else 1  # 0=Pedri, 1=Others
            xtl = float(box.get("xtl"))
            ytl = float(box.get("ytl"))
            xbr = float(box.get("xbr"))
            ybr = float(box.get("ybr"))
            x, y, w, h = convert_box((width, height), (xtl, ytl, xbr, ybr))
            f.write(f"{label} {x} {y} {w} {h}\n")

In [2]:
import shutil
import random

img_dir = "D:/pedri-dataset/images"
label_dir = "D:/pedri-dataset/labels"
split_dirs = {
    "train": "D:/pedri-dataset/images/train",
    "val": "D:/pedri-dataset/images/val",
    "test": "D:/pedri-dataset/images/test"
}
label_split_dirs = {
    "train": "D:/pedri-dataset/labels/train",
    "val": "D:/pedri-dataset/labels/val",
    "test": "D:/pedri-dataset/labels/test"
}

# Create directories
for split in split_dirs.values():
    os.makedirs(split, exist_ok=True)
for split in label_split_dirs.values():
    os.makedirs(split, exist_ok=True)

# List images
images = [f for f in os.listdir(img_dir) if f.endswith(".jpg")]
random.shuffle(images)

# Split: 70% train, 20% val, 10% test
n_train = int(0.7 * len(images))  # ~105 images
n_val = int(0.2 * len(images))    # ~30 images
n_test = len(images) - n_train - n_val  # ~16 images

train_imgs = images[:n_train]
val_imgs = images[n_train:n_train + n_val]
test_imgs = images[n_train + n_val:]

# Move files
for img in train_imgs:
    shutil.move(f"{img_dir}/{img}", f"{split_dirs['train']}/{img}")
    shutil.move(f"{label_dir}/{img.replace('.jpg', '.txt')}", f"{label_split_dirs['train']}/{img.replace('.jpg', '.txt')}")
for img in val_imgs:
    shutil.move(f"{img_dir}/{img}", f"{split_dirs['val']}/{img}")
    shutil.move(f"{label_dir}/{img.replace('.jpg', '.txt')}", f"{label_split_dirs['val']}/{img.replace('.jpg', '.txt')}")
for img in test_imgs:
    shutil.move(f"{img_dir}/{img}", f"{split_dirs['test']}/{img}")
    shutil.move(f"{label_dir}/{img.replace('.jpg', '.txt')}", f"{label_split_dirs['test']}/{img.replace('.jpg', '.txt')}")

In [3]:
import yaml

# Define paths based on your directory structure
train_path = "D:/pedri-dataset/images/train"
val_path = "D:/pedri-dataset/images/val"
test_path = "D:/pedri-dataset/images/test"

# Write the data.yaml file
with open("D:/pedri-dataset/data.yaml", "w") as file:
    yaml.dump({
        "train": train_path,
        "val": val_path,
        "test": test_path,
        # Number of classes
        "nc": 2,  # Two classes: Pedri and Others
        # Class names
        "names": ["Pedri", "Others"]  # List of class names
    }, stream=file, default_flow_style=None)